# Asset Type 1

## Examples, Samples, Use cases:

* Government bond with defined coupon payments
* Digital currency (eRupee)
* Limited edition sovereign
* ...

## Attributes:

* Transferable ✅
* Quantifiable ✅

## Tests:

### Initialisation:

The initialisation is for initialise the python scripts in the notebook get it connected to CorDapp. \
Expected to see a _base directory_ path.

In [1]:
import os
import sys
import math
import time
import IPython
import logging
import warnings
import datetime
import pandas as pd
import numpy  as np
import webbrowser
import pprint as pp
from random   import randrange
from notebook import notebookapp
from IPython.display import display
from IPython.core.display import HTML

# Initialise the notebook settings
def init_notebook():
    sys.path.insert(1, os.path.join(sys.path[0], '{}/lib/'.format(os.environ['BaseDir'])))
    display(HTML("<style>.container { width:98% !important; }</style>"))
    %matplotlib inline
    
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_colwidth', None)

# Initialise the Corda nodes
def init_cordaNodes():
    from corda5Interface import Corda5
    apiServer='https://localhost:8888/api/v1/flow'
    apiServerAuth=('admin', 'admin')
    package = 'com.r3.developers.configurableInstrument'
    replace_maters_list = { 'API_SERVER': apiServer, 'PACKAGE': package}

    nodes = {
            'Authority' : {'x500': 'CN=Authority, OU=Test Dept, O=R3, L=London, C=GB' ,'hash': 'A9C9500BE2EA' },
            'Bob'       : {'x500': 'CN=Bob, OU=Test Dept, O=R3, L=London, C=GB'       ,'hash': '6FBD428F57CF' },
            'Charlie'   : {'x500': 'CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB'   ,'hash': '4EE6FF9D5086' },
            'Alice'     : {'x500': 'CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'     ,'hash': '35D00CE32C92' },
            }

    h = Corda5(level=logging.INFO, nodes=nodes, apiServer=apiServer, apiServerAuth=apiServerAuth, version=2)

    log = h.log
    log.info("Base directory {}".format(os.environ['BaseDir']))
    
    return h

init_notebook()
h = init_cordaNodes()

 18-05 23:28:31 | I | Base directory /Users/zibo.wu/Desktop/configurableLedgerAsset/python3


### Issue:

* Issuer gets to decide the name, transferable or not, expiry or not, redeemable or not, and can add multiple/additional attributes (HashMap) on the same State/Asset.
* Applications like Government Bond, will be transferable, with maturity(some expiry), and redeemable (cash the principal on maturity), with additional attribute of coupon payment dates.
* Applications like College University Degree, will not be transferable, no expiry, and not redeemable. Will have additional attributes like Collage Name, University Name, Education Stream etc etc along with Issuer and Owner(Holder).
* Applications like Limited Edition Sovereign, will be transferable, no expiry but will be redeemable directly with Issuer.

In Asset type 1, all the issuing should be succeed, with **Transferable** and **Quantifiable** assets.

In [2]:
# Issue Example 1 -- Government Bond with defined coupon payments -- Transferable, Quantifiable Instrument
# Expected -- succeed

(req_id, response, return_val_uuid) = h.issue('Authority', 'Alice', 'Government Bond 2024', 
        quantity=10,
        transferable=True,
        expiry=None,
        verifiable=True,
        attributes={'payments': "['10Jun2023', '10Sep2023', '10Dec2023', '10Mar2024']"})

h.message([req_id, response, return_val_uuid])

 18-05 23:28:36 | I | Issuing instrument='Government Bond 2024' to 'Alice'
 18-05 23:28:36 | I | Running action 'Issue' on 'Authority/A9C9500BE2EA/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 18-05 23:29:24 | I | 19926169-bb10-40ea-a6f7-fd7029815d3f
 18-05 23:29:24 | I | Time taken = 0:00:48.372501


16934.1684448916.2009878<Response [200]>19926169-bb10-40ea-a6f7-fd7029815d3f

In [3]:
# Issue Example 2 -- Digital currency (eRupee) -- Transferable, Quantifiable Instrument
# Expected -- succeed

(req_id, response, return_val_uuid) = h.issue('Authority', 'Alice', 'eRupee',
        quantity=100001,
        transferable=True,
        expiry=None,
        verifiable=True
       )

h.message([req_id, response, return_val_uuid])

 18-05 23:29:28 | I | Issuing instrument='eRupee' to 'Alice'
 18-05 23:29:28 | I | Running action 'Issue' on 'Authority/A9C9500BE2EA/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 18-05 23:30:12 | I | 0af2f96b-45f3-4fbd-a7e2-f43068859c5e
 18-05 23:30:12 | I | Time taken = 0:00:44.278233


16934.1684448968.549254<Response [200]>0af2f96b-45f3-4fbd-a7e2-f43068859c5e

### Query:

Query is used for query the instruments that one/all held.

In [4]:
# Query Exmaple 1 -- Query all the instruments

h.query_all_nodes()

 18-05 23:30:45 | I | Result for query=ListInstrument from 'Authority/A9C9500BE2EA/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.076910)


,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,19926169-bb10-40ea-a6f7-fd7029815d3f,Government Bond 2024,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",10,True,None,True,"{'payments': '['10Jun2023', '10Sep2023', '10Dec2023', '10Mar2024']'}"
1,0af2f96b-45f3-4fbd-a7e2-f43068859c5e,eRupee,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",100001,True,None,True,{}


 18-05 23:30:49 | I | Result for query=ListInstrument from 'Bob/6FBD428F57CF/CN=Bob, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.072193)


""


 18-05 23:30:53 | I | Result for query=ListInstrument from 'Charlie/4EE6FF9D5086/CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.073546)


""


 18-05 23:30:57 | I | Result for query=ListInstrument from 'Alice/35D00CE32C92/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.070701)


,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,19926169-bb10-40ea-a6f7-fd7029815d3f,Government Bond 2024,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",10,True,None,True,"{'payments': '['10Jun2023', '10Sep2023', '10Dec2023', '10Mar2024']'}"
1,0af2f96b-45f3-4fbd-a7e2-f43068859c5e,eRupee,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",100001,True,None,True,{}


In [5]:
# Query Exmaple 2 -- Query Alice's instruments

h.query('Alice')

,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,19926169-bb10-40ea-a6f7-fd7029815d3f,Government Bond 2024,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",10,True,None,True,"{'payments': '['10Jun2023', '10Sep2023', '10Dec2023', '10Mar2024']'}"
1,0af2f96b-45f3-4fbd-a7e2-f43068859c5e,eRupee,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",100001,True,None,True,{}


### Transfer:

* Not all instruments are transferable.
* If transferable -- will work on the basic nature of the instrument decided right from the issue point(by the issuer).
* Check = not expired apart from being owned. 

In Asset type 1:
* **Full** transfer from a **holder** to a holder should be **succeed**.
* **Partial** transfer from a **holder** to a holder should be **succeed**.
* Any transfer from a **issuer** to a holder should be **failed**.
* Any transfer from a **non holder** to a holder should be **failed**.

In [6]:
# Transfer Example 1 -- Partial transfer from Alice to Bob
# Expected -- succeed

(req_id, response, return_val) = h.transfer(return_val_uuid, 'Alice', 'Bob', quantity=1000)

h.message([req_id, response, return_val])

 18-05 23:31:07 | I | Tranfering instrument id '0af2f96b-45f3-4fbd-a7e2-f43068859c5e' to 'Bob'
 18-05 23:31:07 | I | Running action 'Transfer' on 'Alice/35D00CE32C92/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'
 18-05 23:32:12 | I | net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@5dbbd018
 18-05 23:32:12 | I | Time taken = 0:01:04.399015


16934.1684449067.941361<Response [200]>net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@5dbbd018

In [7]:
# Transfer Example 2 -- Full transfer from Alice to Charlie
# Expected -- succeed

(req_id, response, return_val) = h.transfer(return_val_uuid, 'Alice', 'Charlie')

h.message([req_id, response, return_val])

 18-05 23:32:18 | I | Tranfering instrument id '0af2f96b-45f3-4fbd-a7e2-f43068859c5e' to 'Charlie'
 18-05 23:32:18 | I | Running action 'Transfer' on 'Alice/35D00CE32C92/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'
 18-05 23:33:26 | I | net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@687e2ac8
 18-05 23:33:26 | I | Time taken = 0:01:08.410693


16934.1684449138.349611<Response [200]>net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@687e2ac8

In [8]:
# Now, if we query all nodes

h.query_all_nodes()

 18-05 23:33:39 | I | Result for query=ListInstrument from 'Authority/A9C9500BE2EA/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.081077)


,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,19926169-bb10-40ea-a6f7-fd7029815d3f,Government Bond 2024,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",10,True,None,True,"{'payments': '['10Jun2023', '10Sep2023', '10Dec2023', '10Mar2024']'}"
1,0af2f96b-45f3-4fbd-a7e2-f43068859c5e,eRupee,"CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",99001,True,None,True,{}
2,bf9260e7-98d7-4d97-a298-dc4777e35110,eRupee,"CN=Bob, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",1000,True,None,True,{}


 18-05 23:33:43 | I | Result for query=ListInstrument from 'Bob/6FBD428F57CF/CN=Bob, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.075483)


,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,bf9260e7-98d7-4d97-a298-dc4777e35110,eRupee,"CN=Bob, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",1000,True,None,True,{}


 18-05 23:33:47 | I | Result for query=ListInstrument from 'Charlie/4EE6FF9D5086/CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.073225)


,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,0af2f96b-45f3-4fbd-a7e2-f43068859c5e,eRupee,"CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",99001,True,None,True,{}


 18-05 23:33:51 | I | Result for query=ListInstrument from 'Alice/35D00CE32C92/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.066075)


,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,19926169-bb10-40ea-a6f7-fd7029815d3f,Government Bond 2024,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",10,True,None,True,"{'payments': '['10Jun2023', '10Sep2023', '10Dec2023', '10Mar2024']'}"


In [9]:
# Transfer Example 3 -- Transfer attempt from the Issuer
# Expected -- failed

(req_id, response, return_val) = h.transfer(return_val_uuid, 'Authority', 'Charlie', quantity=100)

h.message([req_id, response, return_val])

 18-05 23:33:51 | I | Tranfering instrument id '0af2f96b-45f3-4fbd-a7e2-f43068859c5e' to 'Charlie'
 18-05 23:33:51 | I | Running action 'Transfer' on 'Authority/A9C9500BE2EA/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 18-05 23:33:55 | I | {'type': 'FLOW_FAILED', 'message': 'Not able to find a owning state with id 0af2f96b-45f3-4fbd-a7e2-f43068859c5e'}
 18-05 23:33:55 | I | Time taken = 0:00:04.054435


16934.1684449231.848772<Response [200]>{'type': 'FLOW_FAILED', 'message': 'Not able to find a owning state with id 0af2f96b-45f3-4fbd-a7e2-f43068859c5e'}

In [10]:
# Transfer Example 4 -- Transfer attempt from a non-holder Alice
# Expected -- failed

(req_id, response, return_val) = h.transfer(return_val_uuid, 'Alice', 'Bob', quantity=100)

h.message([req_id, response, return_val])

 18-05 23:34:04 | I | Tranfering instrument id '0af2f96b-45f3-4fbd-a7e2-f43068859c5e' to 'Bob'
 18-05 23:34:04 | I | Running action 'Transfer' on 'Alice/35D00CE32C92/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'
 18-05 23:34:08 | I | {'type': 'FLOW_FAILED', 'message': 'Not able to find a owning state with id 0af2f96b-45f3-4fbd-a7e2-f43068859c5e'}
 18-05 23:34:08 | I | Time taken = 0:00:04.060973


16934.1684449244.574858<Response [200]>{'type': 'FLOW_FAILED', 'message': 'Not able to find a owning state with id 0af2f96b-45f3-4fbd-a7e2-f43068859c5e'}

### Redeem:

* Not all instruments will have expiry/maturity.
* Redeem can be executed by either the owner (holder of the asset) or also by the issuer of the asset (present design, can be controlled)
* If permitted, it will work on the basic nature of the instrument decided right from the issue point(by the issuer).

In Asset type 1, all the redeems apart from non holders should be **succeed**

In [11]:
# Redeem Example 1 -- Partial redeem by the holder
# Expected -- succeed

h.redeem(return_val_uuid, 'Charlie', quantity=1000)

 18-05 23:34:12 | I | Redeeming instrument id '0af2f96b-45f3-4fbd-a7e2-f43068859c5e' quantity '1000'
 18-05 23:34:12 | I | Running action 'Redeem' on 'Charlie/4EE6FF9D5086/CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB'
 18-05 23:35:00 | I | net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@40d9b254
 18-05 23:35:00 | I | Time taken = 0:00:48.299488


('16934.1684449252.521526',
 <Response [200]>,
 'net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@40d9b254')

In [12]:
# Now, if we query Charlie again

h.query('Charlie')

,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,0af2f96b-45f3-4fbd-a7e2-f43068859c5e,eRupee,"CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",98001,True,None,True,{}


In [13]:
# Redeem Example 2 -- Full redeem by the holder
# Expected -- succeed

h.redeem(return_val_uuid, 'Charlie')

 18-05 23:35:39 | I | Redeeming instrument id '0af2f96b-45f3-4fbd-a7e2-f43068859c5e' quantity 'None'
 18-05 23:35:39 | I | Running action 'Redeem' on 'Charlie/4EE6FF9D5086/CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB'
 18-05 23:36:27 | I | net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@6a795b70
 18-05 23:36:27 | I | Time taken = 0:00:48.274717


('16934.1684449339.1107452',
 <Response [200]>,
 'net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@6a795b70')

In [14]:
# Now, if we query Charlie again

h.query('Charlie')

""


In [14]:
# Redeem Example 3 -- Partial redeem from Bob by the Issuer
# Expected -- succeed

h.redeem('bf9260e7-98d7-4d97-a298-dc4777e35110', 'Authority', quantity=500)

 18-05 23:36:59 | I | Redeeming instrument id 'bf9260e7-98d7-4d97-a298-dc4777e35110' quantity '500'
 18-05 23:36:59 | I | Running action 'Redeem' on 'Authority/A9C9500BE2EA/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 18-05 23:37:43 | I | net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@6183909a
 18-05 23:37:43 | I | Time taken = 0:00:44.320398


('16934.1684449419.211481',
 <Response [200]>,
 'net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@6183909a')

In [15]:
# Now, if we query Bob

h.query('Bob')

,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,bf9260e7-98d7-4d97-a298-dc4777e35110,eRupee,"CN=Bob, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",500,True,None,True,{}


In [16]:
# Redeem Example 4 -- Full redeem from Bob by the Issuer
# Expected -- succeed

h.redeem('bf9260e7-98d7-4d97-a298-dc4777e35110', 'Authority')

 18-05 23:38:15 | I | Redeeming instrument id 'bf9260e7-98d7-4d97-a298-dc4777e35110' quantity 'None'
 18-05 23:38:15 | I | Running action 'Redeem' on 'Authority/A9C9500BE2EA/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'
 18-05 23:38:56 | I | net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@7e40328a
 18-05 23:38:56 | I | Time taken = 0:00:40.255724


('16934.1684449495.988829',
 <Response [200]>,
 'net.corda.ledger.utxo.flow.impl.FinalizationResultImpl@7e40328a')

In [17]:
# Now, if we query Bob again

h.query('Bob')

""


In [18]:
# Now, if we query all nodes

h.query_all_nodes()

 18-05 23:39:12 | I | Result for query=ListInstrument from 'Authority/A9C9500BE2EA/CN=Authority, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.073291)


,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,19926169-bb10-40ea-a6f7-fd7029815d3f,Government Bond 2024,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",10,True,None,True,"{'payments': '['10Jun2023', '10Sep2023', '10Dec2023', '10Mar2024']'}"


 18-05 23:39:16 | I | Result for query=ListInstrument from 'Bob/6FBD428F57CF/CN=Bob, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.067306)


""


 18-05 23:39:20 | I | Result for query=ListInstrument from 'Charlie/4EE6FF9D5086/CN=Charlie, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.065281)


""


 18-05 23:39:24 | I | Result for query=ListInstrument from 'Alice/35D00CE32C92/CN=Alice, OU=Test Dept, O=R3, L=London, C=GB'. (Time taken=0:00:04.064416)


,id,name,owner,issuer,quantity,transferable,expiry,verifiable,attributes
0,19926169-bb10-40ea-a6f7-fd7029815d3f,Government Bond 2024,"CN=Alice, OU=Test Dept, O=R3, L=London, C=GB","CN=Authority, OU=Test Dept, O=R3, L=London, C=GB",10,True,None,True,"{'payments': '['10Jun2023', '10Sep2023', '10Dec2023', '10Mar2024']'}"


In [19]:
# Redeem Example 5 -- Redeem from non holder
## Expected -- failed

h.redeem('19926169-bb10-40ea-a6f7-fd7029815d3f', 'Bob')

 18-05 23:39:33 | I | Redeeming instrument id '19926169-bb10-40ea-a6f7-fd7029815d3f' quantity 'None'
 18-05 23:39:33 | I | Running action 'Redeem' on 'Bob/6FBD428F57CF/CN=Bob, OU=Test Dept, O=R3, L=London, C=GB'
 18-05 23:39:38 | I | {'type': 'FLOW_FAILED', 'message': 'Not able to find a owning state with id 19926169-bb10-40ea-a6f7-fd7029815d3f'}
 18-05 23:39:38 | I | Time taken = 0:00:04.052630


('16934.1684449573.9691641',
 <Response [200]>,
 {'type': 'FLOW_FAILED',
  'message': 'Not able to find a owning state with id 19926169-bb10-40ea-a6f7-fd7029815d3f'})

### Print to PDF:

"Print to PDF" is used for printing any instrument with a specific format.

In [20]:
# Print to PDF Example 1 -- Print the Government Bond holding by Alice
## Expected -- succeed

pdf_report = h.print_to_pdf('Alice', '19926169-bb10-40ea-a6f7-fd7029815d3f', show=False)
webbrowser.open("file:///{}".format(pdf_report))

 18-05 23:40:35 | I | Time taken = 0:00:08.337942


True

In [21]:
# Print to PDF Example 2 -- Print the Government Bond holding by Issuer
## Expected -- succeed

pdf_report = h.print_to_pdf('Authority', '19926169-bb10-40ea-a6f7-fd7029815d3f', show=False)
webbrowser.open("file:///{}".format(pdf_report))

 18-05 23:40:58 | I | Time taken = 0:00:07.512451


True

In [22]:
# Print to PDF Example 3 -- Print the Government Bond holding by non holder
## Expected -- failed

pdf_report = h.print_to_pdf('Charlie', '19926169-bb10-40ea-a6f7-fd7029815d3f', show=False)
webbrowser.open("file:///{}".format(pdf_report))

KeyError: 'id'